In [ ]:
!pip install transformers
!pip install pytorch-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 947.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 110.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filena

In [ ]:
import torch
import torch.cuda
from google.colab import drive
drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {'cuda' if torch.cuda.is_available() else 'cpu'}") 

Mounted at /content/drive
Using cuda


In [ ]:
import pandas as pd
import torch
from pytorch_transformers import BertTokenizer
from pytorch_transformers import BertModel
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
labelsX = {'NONE': 0,
         'HATE': 1,
         'PRFN': 2,
         'OFFN': 3}

def preTesting(dataInput):
    data = []
    label = []
 
    for n in dataInput.iloc:
        n['tweet'].lower()
        n['tweet'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '') 
        n['tweet'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '')
        n['tweet'].replace('[^\w\s]', '')
        n['tweet'].replace('\d', '')
        n['tweet'].replace(r'[^\x00-\x7F]+', r' ') 

        data.append(n['tweet'])
        label.append(labelsX[n['task_2']])

    return data, label

data1 = pd.read_table('/content/drive/MyDrive/deepNN-labs/data/HASOCData/english_dataset.tsv')
tweets, labels = preTesting(data1)


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bertweet = bertweet.to(device)

# Create a custom dataset class
class TweetDataset(Dataset):
    def __init__(self, tweets, labels):
        self.tweets = tweets
        self.labels = labels

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
        return self.tweets[idx], self.labels[idx]

# Tokenize tweets and obtain input IDs

encoded_tweets = tokenizer(tweets, padding=True, truncation=True, return_tensors="pt")
input_ids = encoded_tweets["input_ids"]




emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch_size = 10
num_batches = len(input_ids) // batch_size
embeddings = []
with torch.no_grad():
    bertweet.eval()  # Set model to evaluation mode
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_input_ids = input_ids[start_idx:end_idx].to(device)

        bertweet_output = bertweet(batch_input_ids)
        batch_embeddings = bertweet_output.last_hidden_state

        embeddings.append(batch_embeddings)

    # Compute embeddings for the remaining tokens
    if len(input_ids) % batch_size != 0:
        start_idx = num_batches * batch_size
        batch_input_ids = input_ids[start_idx:].to(device)

        bertweet_output = bertweet(batch_input_ids)
        batch_embeddings = bertweet_output.last_hidden_state

        embeddings.append(batch_embeddings)

embeddings = torch.cat(embeddings, dim=0)

In [ ]:
from torch.autograd import Variable
# Convert input IDs to BERTweet emb


# Create train and validation dataset and dataloaders
all_data = TweetDataset(embeddings, labels)

train_samp = int(0.8*len(all_data))
val_samp = len(all_data) - train_samp
train, val = torch.utils.data.random_split(all_data, [train_samp, val_samp], generator=torch.Generator())


train_loader = DataLoader(train, batch_size=32, shuffle=True)
val_loader = DataLoader(val, batch_size=32, shuffle=False)

# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMClassifier, self).__init__()
        self.output_dim = output_dim #nr of classes
        self.num_layers = num_layers #nr of layers
        self.input_dim = input_dim #input size
        self.hidden_dim = hidden_dim #hidden state
        #self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.5)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
      #  lstm_out, _ = self.lstm(x)
       # out = self.fc(lstm_out[:, -1, :])
        #return out
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_dim)).to(device)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_dim)).to(device)

        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_dim)
        out = self.relu(hn)
        out = self.fc1(out)
        out = self.fc(out)
        return out


input_dim = 768
hidden_dim = 128
num_layers = 1
output_dim = 4

model = LSTMClassifier(input_dim, hidden_dim, num_layers, output_dim).to(device)

# Train and evaluate the model
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.001)
to_onehot = nn.Embedding(4,4)
to_onehot.weight.data = torch.eye(4)

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        target = to_onehot(target)
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        #print(output, "\n Taer: ",target)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    

    valid_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        model.eval()
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()

            # Calculate accuracy
            predictions = torch.argmax(output, dim=1)
            # print("härhärhär:\n", predictions, "perperper:\n", (predictions == target).sum().item(), "korkorkor:\n", target)
            correct += (predictions == target).sum().item()
            total += target.size(0)

    valid_accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {valid_loss / len(val_loader)}, Validation Accuracy: {valid_accuracy}")


In [ ]:
def preprocess(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['tweet'] = data['tweet'].str.lower()
    data['tweet'] = data['tweet'].str.replace(r'(@\S+)', r' ', regex=True)
    data['tweet'] = data['tweet'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['tweet'] = data['tweet'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['tweet'] = data['tweet'].str.replace('[^\w\s]', '')                                                       # remove special characters
    data['tweet'] = data['tweet'].replace('\d', '', regex=True)
    data['tweet'] = data['tweet'].str.replace(r'[^\x00-\x7F]+', r' ', regex=True) 
    
    i = 0
    for index, row in data.iterrows():
        text = row['tweet']
        #word_tokens = tokenizer.tokenize(text)
        token_id = tokenizerTweet.encode(text)
        #filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_ = df_.append({
            "index": i,
            "Class": row['task_2'],
            "Tweet": text,
            'Token_id': token_id
        }, ignore_index=True)
        i = i + 1
    return df_

In [ ]:
  bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
  bertweet = bertweet.to(device)
tokenizerTweet= AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

data1 = pd.read_table('/content/drive/MyDrive/deepNN-labs/data/HASOCData/english_dataset.tsv')
data2 = pd.read_table('/content/drive/MyDrive/deepNN-labs/data/HASOCData/hasoc2019_en_test-2919.tsv')
columns = ['index', 'Tweet', 'Token_id','Class']
data = preprocess(data1, columns)

dataT = preprocess(data2, columns)


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
for _, n in data.iterrows():
  print(n['Token_id'])

In [ ]:
training_data = []

labels = {'NONE': 0,
         'HATE': 1,
         'PRFN': 2,
         'OFFN': 3}
#embed size 768

for _, n in data.iterrows():
    #print(n['index'])
    input_ids = torch.tensor([n['Token_id']]).to(device)
    #print(input_ids)
    with torch.no_grad():
        features = bertweet(input_ids)
    embeds = features[-1]

    
    embeds = embeds.squeeze(0)
    print(embeds.shape)
    training_data.append((embeds, labels[n['Class']]))

In [ ]:
training_data[0][0].squeeze(0).shape

torch.Size([768])

In [ ]:
training_data = []
labels = {'NONE': 0,
         'HATE': 1,
         'PRFN': 2,
         'OFFN': 3}
#embed size 768

for _, n in data.iterrows():
    #print(n['index'])
    input_ids = torch.tensor([n['Token_id']]).to(device)
    #print(input_ids)
    with torch.no_grad():
        features = bertweet(input_ids)
    embeds = features[-2]
    padd_size = (120 - len(n['Token_id']))
    
    #print(padd_size + len(n['Token_id']))

    padd_tens = torch.zeros((1, padd_size, 768))
    padd_tens = padd_tens.to(device)

    new_tens = torch.cat((embeds, padd_tens), dim=1)
    new_tens = new_tens.squeeze(0)

    training_data.append((new_tens, labels[n['Class']]))

testing_data = []

labels = {'NONE': 0,
         'HATE': 1,
         'PRFN': 2,
         'OFFN': 3}
#embed size 768

for _, n in dataT.iterrows():
    #print(n['index'])
    input_ids = torch.tensor([n['Token_id']]).to(device)
    #print(input_ids)
    with torch.no_grad():
        features = bertweet(input_ids)
    embeds = features[-2]
    padd_size = (120 - len(n['Token_id']))
    
    print(padd_size + len(n['Token_id']), " ", n['index'])

    padd_tens = torch.zeros((1, padd_size, 768))
    padd_tens = padd_tens.to(device)

    new_tens = torch.cat((embeds, padd_tens), dim=1)
    new_tens = new_tens.squeeze(0)

    testing_data.append((new_tens, labels[n['Class']]))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(120, 128, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(128 * 384, 120)  # Adjust this dimension based on the output size from the previous layer
        self.fc2 = nn.Linear(120, 4)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        #x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

In [ ]:

def train_model(model, criterion, optimizer, lista, val, num_epochs):
    bestValLoss = 1000
    for epochs in range(num_epochs):
        loss = 0
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        running_loss2 = 0.0
        correct_predictions2 = 0
        total_predictions2 = 0

        for batch_nr, (data, label) in enumerate(lista):
            #print("Data, ", data, "\n Labels: ", labels, "\n batch: ", batch_nr)
            prediction = model.forward(data)
            _, predicted = torch.max(prediction, 1)

            #label = to_onehot(label).to(device)
            label = label.to(device)
            loss = criterion(prediction, label)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
            
            running_loss += loss.item() * data.size(0)
            correct_predictions += (predicted == label).sum().item()
            total_predictions += label.size(0)
        
        ##Validation
        for batch_nr, (data, label) in enumerate(lista):
            #print("Data, ", data, "\n Labels: ", labels, "\n batch: ", batch_nr)
            prediction = model.forward(data)
            _, predicted = torch.max(prediction, 1)

            label = label.to(device)
            loss = criterion(prediction, label)

            if (loss < bestValLoss):
                bestValLoss = loss
                torch.save(model, f="./test")
              
            running_loss2 += loss.item() * data.size(0)
            correct_predictions2 += (predicted == label).sum().item()
            total_predictions2 += label.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = correct_predictions / total_predictions

        epoch_loss2 = running_loss2 / len(val_loader.dataset)
        epoch_acc2 = correct_predictions2 / total_predictions2

        print(f"Epoch {epochs+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

        print(f"Epoch {epochs+1}/{num_epochs}, Val_Loss: {epoch_loss2:.4f}, Val_Acc: {epoch_acc2:.4f}")

def test(model, testData):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in testData:
            outputs = model.forward(data)
            _, pred = torch.max(outputs, 1)
            label = labels.to(device)
            correct += (pred == label).sum().item()
            total += label.size(0)

    print(correct/total)

In [ ]:
LEARNING_RATE = 0.0001
to_onehot = nn.Embedding(4,4)
to_onehot.weight.data = torch.eye(4)
# Load our network
model = Net()
model = model.to(device)

# Define our loss function
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

  


train_samp = int(0.8*len(training_data))
val_samp = len(training_data) - train_samp

train, val = torch.utils.data.random_split(training_data, [train_samp, val_samp], generator=torch.Generator())

dataloader = DataLoader(train, batch_size=32, shuffle=True)

val_loader = DataLoader(val, batch_size=32, shuffle=True)


# Define our optimizer

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay=0.005)

# Train the model
msg = "trainloss leaky relu and adam"
trained_model = train_model(model, criterion, optimizer, dataloader, val, 40)


In [ ]:
tester = DataLoader(testing_data, batch_size=32, shuffle=False)
test(model, tester)

0.7259323503902863


In [ ]:
def embedder(text):
    
    token_ids = tokenizerTweet.encode(text)
    #print(token_ids)
    input_ids = torch.tensor([token_ids]).to(device)
    #print(input_ids)
    with torch.no_grad():
        features = bertweet(input_ids)
    embeds = features[-2]

    padd_size = (90 - len(token_ids))
    

    padd_tens = torch.zeros((1, padd_size, 768))
    padd_tens = padd_tens.to(device)

    new_tens = torch.cat((embeds, padd_tens), dim=1)
    #new_tens = new_tens.squeeze(0)
    
    return new_tens

t = embedder("Farewell SarahHuckabeeSanders Your lies for the asshole POTUS Will NEVER be missed. GladYourGone FuckTrump")

prediction = model.forward(t)

prediction

tensor([[ -57800.8203,  -67118.3125,  -75211.5625, -108935.4453]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
max = 0
id = 0
st = 0
for _, n in data.iterrows():
    print(len(n['Token_id']))
    if len(n['Token_id']) > max:
      max = len(n['Token_id'])
      id = n['index']
      st = n['Tweet']
print("Max: ", max, " index: ", id)

In [ ]:
class HateSpeechDetector(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(HateSpeechDetector, self).__init__()
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.5)
        self.fc1 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.dropout(lstm_out[:, -1, :])
        out = self.fc1(out)
        out = self.activation(out)
        return out

input_dim = 768  # BERT-base embedding dimension
hidden_dim = 128
num_layers = 2
output_dim = 4

model = HateSpeechDetector(input_dim, hidden_dim, num_layers, output_dim)
model = model.to(device)

In [ ]:
from torch.optim.lr_scheduler import StepLR
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(dataloader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    scheduler.step()
    # Evaluate on the validation set
    model.eval()
    with torch.no_grad():
        valid_loss = 0
        correct = 0
        total = 0
        
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()
            
            # Calculate accuracy
            predictions = torch.argmax(output, dim=1)
            correct += (predictions == target).sum().item()
            total += target.size(0)
            #print("total:", total, " correct ", correct)
            print(predictions)

    valid_accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}, Validation Loss: {valid_loss / len(val_loader)}, Validation Accuracy: {valid_accuracy}")


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,